In [9]:
import pandas as pd
import category_encoders as ce

df_sample = pd.read_csv("/Users/macbook/Documents/GitHub/DSfinProject/dsfinlendclub/data/01_raw/Loan_status_sample.csv", low_memory=False)
df_tot = pd.read_csv("/Users/macbook/Documents/GitHub/DSfinProject/dsfinlendclub/data/01_raw/Loan_status_2007-2020Q3.csv", low_memory=False)
df_sample.head(5)

,Unnamed: 0.1,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,1746494,25450,167338079,4000.0,4000.0,4000.0,36 months,13.08%,134.93,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,2370717,81861,71016917,24000.0,24000.0,24000.0,60 months,9.16%,500.07,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,2264870,397109,39589826,5000.0,5000.0,5000.0,36 months,10.49%,162.49,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,595422,15492,134798709,24000.0,24000.0,24000.0,60 months,11.05%,522.42,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,562657,90591,127097355,14000.0,14000.0,14000.0,60 months,13.59%,322.79,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [2]:
print(df_sample.shape)  # Check dataset dimensions
print(df_sample.info())  # Check data types and missing values

(100000, 143)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 143 entries, Unnamed: 0.1 to debt_settlement_flag
dtypes: float64(106), int64(3), object(34)
memory usage: 109.1+ MB
None


### Step 1: Deduplication based on id field

Initial deduplication check. Create a function 'check_and_remove_duplicates' to set into pipline

In [10]:
# Removing duplicate rows
df_deduped = df_tot.drop_duplicates(inplace=True)

try:
    df_deduped.shape # Note: it will fail if empty (i.e. no duplicates)
except:
    print("No duplicates")

No duplicates


### Step 2: Handling missing values/feature engineering

In [11]:
# Checking for missing values
missing_percentage = (df_tot.isnull().sum() / len(df_tot)) * 100
missing_percentage[missing_percentage > 0].sort_values(ascending=False)

hardship_loan_status       95.097886
hardship_reason            95.090332
hardship_status            95.090229
hardship_dpd               95.090161
payment_plan_start_date    95.090127
                             ...    
last_pymnt_amnt             0.000034
last_fico_range_high        0.000034
last_fico_range_low         0.000034
policy_code                 0.000034
debt_settlement_flag        0.000034
Length: 140, dtype: float64

### Step 3. Remove unwanted columns.
After dataset exploration we will remove specific columns. We will use 'drop_unwanted_columns' method in the pipelines.
We will combine the columns into `columns_to_drop` list.
Columns will be selected according to:
- **business/problem relevance**: is the data relevant to the problem I am trying to solve (e.g. Do I need IDs?)
- **high % of missing values**: especially if the data is not easily recoverable or imputation might introduce significant bias.
- **little to no variance**: provide minimal information for the model and can be dropped.
- **high correlation**: if two or more columns are highly correlated, they contain redundant information. You might drop one of them to reduce 'multicollinearity'.
- **high cardinality**: Categorical columns with too many unique values (high cardinality) can lead to overfitting and increased computational cost. These can be dropped or encoded differently.

In [20]:
def drop_unwanted_columns(x: pd.DataFrame, drop_list=None) -> pd.DataFrame:
    """
    Drops unwanted columns from the DataFrame.

    Parameters:
        :param drop_list: List of column names to drop. Defaults to ['Unnamed: 0.1', 'Unnamed: 0'].
        :param x: df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A new DataFrame with specified columns dropped.

    """
    if drop_list is None:
        drop_list = ['Unnamed: 0.1', 'Unnamed: 0']

    return x.drop(columns=[col for col in drop_list if col in x.columns], axis=1)

# Dropping the columns which do not have a predictive power in most applications
| Column                | Reason to Drop                                                                |
|-----------------------|-------------------------------------------------------------------------------|
| `id`                  | Unique loan identifier — no predictive value                                  |
| `member_id`           | Internal LC user ID — unique, not informative                                 |
| `funded_amnt_inv`     | Duplicate of `funded_amnt`, just investor's portion                           |
| `policy_code`         | Always 1 in public data — constant, no variance                               |
| `url`                 | Link to LC listing page                                                       |
| `title`               | User-entered title for loan purpose — very noisy and redundant with `purpose` |
| `zip_code`            | Partial ZIP only (first 3 digits) — privacy-limited, rarely useful            |
| `emp_title`           | Messy text — you might drop it unless you're doing NLP (?)                    |
| `pymnt_plan`          | Always `"n"` — constant value                                                 |
| `application_type`    | Will be captured by `is_joint_app` flag (will appear below)                   |
| `next_pymnt_d`        | Future date — only useful for post-loan monitoring, not origination modeling  |


In [33]:
# noname columns
delete_no_library_cols = ['Unnamed: 0.1', 'Unnamed: 0']
# administrative columns
descr_cols = [
    'id', 'member_id', 'url', 'desc', 'title',
    'zip_code', 'emp_title', 'policy_code', 'pymnt_plan',
    'application_type', 'funded_amnt_inv', 'next_pymnt_d'
]

columns_to_drop = delete_no_library_cols.copy()
columns_to_drop.extend(descr_cols)

# Apply function from pipeline to delete columns
df_tot = drop_unwanted_columns(df_tot,columns_to_drop)
df_tot.head(10)

,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,...,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,hardship_status_filled,is_joint_app
0,5000.0,5000.0,36 months,10.65%,162.87,B,B2,10+ years,RENT,24000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO_HARDSHIP,0
1,2500.0,2500.0,60 months,15.27%,59.83,C,C4,< 1 year,RENT,30000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO_HARDSHIP,0
2,2400.0,2400.0,36 months,15.96%,84.33,C,C5,10+ years,RENT,12252.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO_HARDSHIP,0
3,10000.0,10000.0,36 months,13.49%,339.31,C,C1,10+ years,RENT,49200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO_HARDSHIP,0
4,3000.0,3000.0,60 months,12.69%,67.79,B,B5,1 year,RENT,80000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO_HARDSHIP,0
5,5000.0,5000.0,36 months,7.90%,156.46,A,A4,3 years,RENT,36000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO_HARDSHIP,0
6,7000.0,7000.0,60 months,15.96%,170.08,C,C5,8 years,RENT,47004.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO_HARDSHIP,0
7,3000.0,3000.0,36 months,18.64%,109.43,E,E1,9 years,RENT,48000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO_HARDSHIP,0
8,5600.0,5600.0,60 months,21.28%,152.39,F,F2,4 years,OWN,40000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO_HARDSHIP,0
9,5375.0,5375.0,60 months,12.69%,121.45,B,B5,< 1 year,RENT,15000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO_HARDSHIP,0


In [32]:
# Set threshold
missing_threshold = 0.8

# Calculate missing value ratio
missing_ratio = df_tot.isnull().mean()

# Filter columns that have less than 80% missing values
columns_to_keep = missing_ratio[missing_ratio < missing_threshold].index.tolist()

# Print the columns that have less than 80% missing values
print("✅ Columns with less than 80% missing:")
print(columns_to_keep)

# Print columns with higher than 80% missing values
columns_to_drop = missing_ratio[missing_ratio >= missing_threshold].index.tolist()
print("\n🚫 Columns with 80% or more missing:")
print(columns_to_drop)

✅ Columns with less than 80% missing:
['loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'tota

### Feature engineering and missing values handling

In [ ]:
# Not all the columns with high values for NaNs are uninformative

# Create hardship flag based on information are there a hardship event (1) or not (0)
df_tot['has_hardship'] = df_tot['hardship_status'].notnull().astype(int)

# Create a flag for joint or individual loan signal
df_tot['annual_inc_final'] = df_tot['annual_inc']
df_tot.loc[df_tot['is_joint_app'] == 1, 'annual_inc_final'] = df_tot['annual_inc_joint']

# Create a debt-to-income ratio for all individual and joined applications
df_tot['dti_final'] = df_tot['dti']
df_tot.loc[df_tot['is_joint_app'] == 1, 'dti_final'] = df_tot['dti_joint']

# Create a feature for verification status combil=ning individual and joint loans
df_tot['verification_status_final'] = df_tot['verification_status']
df_tot.loc[df_tot['is_joint_app'] == 1, 'verification_status_final'] = df_tot['verification_status_joint']

# Create a joint feature for the revolving balance of individual or joint applications
df_tot['revol_bal_final'] = df_tot['revol_bal']
df_tot.loc[df_tot['is_joint_app'] == 1, 'revol_bal_final'] = df_tot['revol_bal_joint']

# Create a flag for a simple fact if hardship started on Late or other loan status
df_tot['was_late_before_hardship'] = df_tot['hardship_loan_status'].str.contains('Late', na=False).astype(int)

# Fill missing values for 'hardship_dpd' with 0 assuming no days past due for non-hardship loans
df_tot['hardship_dpd_filled'] = df_tot['hardship_dpd'].fillna(0)

# Fill the NaNs for the number of months since the borrower's last public derogatory record with high value (999).
df_tot['mths_since_last_record_filled'] = df_tot['mths_since_last_record'].fillna(999)

# String fields conversion

In [ ]:
# Loan term field processing
df_tot['term'] = df_tot['term'].str.extract(r'(\d+)').astype(int)

# 'int_rate' field processing
df_tot['int_rate'] = df_tot['int_rate'].str.rstrip('%').astype(float)
df_tot['int_rate'] = df_tot['int_rate'] / 100

# Lending club classification fields 'grade' and 'sub_grade' (optional to use in modeling but better to encode properly)
grade_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
df_tot['grade_encoded'] = df_tot['grade'].map(grade_mapping)
# Create a mapping from subgrade to ordinal value
subgrades = ['A1', 'A2', 'A3', 'A4', 'A5',
             'B1', 'B2', 'B3', 'B4', 'B5',
             'C1', 'C2', 'C3', 'C4', 'C5',
             'D1', 'D2', 'D3', 'D4', 'D5',
             'E1', 'E2', 'E3', 'E4', 'E5',
             'F1', 'F2', 'F3', 'F4', 'F5',
             'G1', 'G2', 'G3', 'G4', 'G5']

subgrade_mapping = {sub: i + 1 for i, sub in enumerate(subgrades)}
df_tot['sub_grade_encoded'] = df_tot['sub_grade'].map(subgrade_mapping)

# Employment length field 'emp_length' processing to numerical values
emp_length_mapping = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
    'n/a': None  # or np.nan
}

df_tot['emp_length_clean'] = df_tot['emp_length'].map(emp_length_mapping)

# We will keep one of the fields 'emp_length_clean_tree' or 'emp_length_clean_reg' for different model types. Field 'emp_length' safe to delete at the end of data preparation step, see delete_model_related

# Filling missing with -1 (Tree-based (RF, XGBoost, LGBM))
df_tot['emp_length_clean_tree'] = df_tot['emp_length_clean'].fillna(-1)

# Filling missing with median for Logistic / Linear / SVM
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
df_tot[['emp_length_clean_reg']] = imputer.fit_transform(df_tot[['emp_length_clean']])

# Columns to drop if using tree related models RF, XGBoost, LGBM
delete_model_related_emp_length_tree = ['emp_length','emp_length_clean', 'emp_length_clean_reg']

# Columns to drop if using regressions Logistic / Linear / SVM
delete_model_related_emp_length_reg = ['emp_length', 'emp_length_clean', 'emp_length_clean_tree']


# Creating a fork with two datasets with different 'purpose' encoding: (i) with one-hot encoding and (ii) target encoding

In [ ]:
# --------------------------------------------
# ONE-HOT ENCODING VERSION
# --------------------------------------------
df_tot_onehot_purpose = df_tot.copy()
df_tot_onehot_purpose = pd.get_dummies(df_tot_onehot_purpose, columns=['purpose'], drop_first=True)

In [ ]:
# OPTION - using target encoding as alternative (not together with one-hot encoding)
# apply using cross-validation folds (e.g. KFold) when evaluating model performance.
# --------------------------------------------
# TARGET ENCODING VERSION
# --------------------------------------------
# Create a fresh copy
df_tot_target_purpose = df_tot.copy()

# Creating binary target column
df_tot_target_purpose['loan_status_binary'] = df_tot_target_purpose['loan_status'].map({
    'Fully Paid': 0,
    'Charged Off': 1,
    'Default': 1
})

# Filtering out rows with NaN target (e.g., 'Current', 'Late', etc.) for a training
df_tot_target_purpose = df_tot_target_purpose[df_tot_target_purpose['loan_status_binary'].notnull()].copy()

# Applying target encoding to 'purpose'
encoder = ce.TargetEncoder(cols=['purpose'])
df_tot_target_purpose['purpose_encoded'] = encoder.fit_transform(
    df_tot_target_purpose['purpose'],
    df_tot_target_purpose['loan_status_binary']
)

# (Optional) Drop original 'purpose' column
# df_tot_target_purpose.drop(columns=['purpose'], inplace=True)

In [15]:
# Delete the columns which we use to create additional combined features to reduce noise and redundancy. Keeping future modelling simple we delete all columns which give more detailed description of a hardship period or to detailed description of co-borrower credit quality.
delete_flag_related = ['dti', 'dti_joint', 'annual_inc', 'annual_inc_joint', 'hardship_status', 'hardship_type', 'hardship_reason', 'hardship_start_date', 'hardship_end_date', 'hardship_amount', 'hardship_length', 'deferral_term','verification_status', 'verification_status_joint', 'revol_bal', 'revol_bal_joint', 'hardship_loan_status', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 'orig_projected_additional_accrued_interest', 'payment_plan_start_date', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'grade', 'sub_grade']
columns_to_drop.extend(delete_flag_related)

In [99]:
df_sample['id'].value_counts()



id
167338079    1
693399       1
119962746    1
161374059    1
74591293     1
            ..
306673       1
74592806     1
131409111    1
139313158    1
148437941    1
Name: count, Length: 100000, dtype: int64

# Job title processing

In [53]:

import logging
logging.getLogger().setLevel(logging.ERROR)
%load_ext kedro.ipython

ref_job_families = catalog.load("emp_title_mapping")
df_sample = catalog.load("sample")

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


[03/26/25 11:15:47] INFO     Loading data from emp_title_mapping (CSVDataset)...                ]8;id=64494;file:///Users/macbook/tensorflow-m1/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=337840;file:///Users/macbook/tensorflow-m1/lib/python3.8/site-packages/kedro/io/data_catalog.py#539\539]8;;\

Number of existing job families: 32


In [76]:
# cleaning the job category dataset
ref_job_families["Job Title"] = ref_job_families["Job Title"].str.strip().str.lower()
ref_job_families["Category"] = ref_job_families["Category"].str.strip().str.lower()

# Drop duplicates in the job category dataset based only on two columns
ref_job_families_deduped = ref_job_families.drop_duplicates(subset=["Category", "Job Title"])

# Cleaning text data in the dataset (sample)
df_sample['emp_title_clean'] = df_sample['emp_title'].str.lower().str.strip()


In [83]:
ref_job_families_deduped.head()

,Job Title,Category
0,web editor,content/media
1,route manager,operations
2,quality manager,quality assurance
3,budget mgr,finance
4,inspector,quality assurance


In [85]:
df_mapped_deduped = pd.merge(left = df_sample
                     , right = ref_job_families_deduped
                     , left_on = "emp_title_clean"
                     , right_on= "Job Title"
                     , how = "left"
                    )
df_mapped_deduped['Category'].value_counts()[:100]


Category
management            3622
education             2287
executive             1755
healthcare            1343
administrative        1010
sales                  766
logistics              615
finance                541
technical support      461
consulting             169
customer service       127
quality assurance      113
operations             109
emergency services     104
marketing               86
labor                   62
insurance               54
it/technology           41
procurement             38
religious               22
manufacturing           19
construction            10
security                10
engineering              6
other                    5
human resources          5
agriculture              4
hospitality              2
content/media            1
event management         1
animal care              1
project management       1
Name: count, dtype: int64

In [61]:
# Cleaning text data
df_sample['emp_title_clean'] = df_sample['emp_title'].str.lower().str.strip()
df_sample['emp_title_clean'].value_counts()[:10]


emp_title_clean
teacher             2121
manager             2062
owner               1245
registered nurse    1019
driver              1008
supervisor           978
sales                765
rn                   758
office manager       670
truck driver         612
Name: count, dtype: int64

In [64]:
ref_job_families_deduped = ref_job_families.drop_duplicates()
df_mapped_deduped = pd.merge(left = df_sample
                     , right = ref_job_families_deduped
                     , left_on = "emp_title_clean"
                     , right_on= "Job Title"
                     , how = "left"
                    )

In [65]:
print(df_sample.shape, df_mapped_deduped.shape)

(100000, 145) (100000, 147)


In [66]:
print(ref_job_families.shape)
print(ref_job_families.drop_duplicates().shape)

(91, 2)
(89, 2)


In [67]:
# The problem can be seen by grouping the ref data by job title and category:
row_count = ref_job_families.groupby(["Category", "Job Title"]).size().reset_index(name='count')
row_count[row_count['count'] > 1]

,Category,Job Title,count
17,Education,teacher,2
35,Healthcare,medical assistant,2


In [72]:
ref_job_families[ref_job_families["Job Title"] == "teacher"]

,Job Title,Category
33,teacher,Education
66,teacher,Education


In [73]:
ref_job_families[ref_job_families["Job Title"] == "medical assistant"]

,Job Title,Category
70,medical assistant,Healthcare
88,medical assistant,Healthcare


In [69]:
ref_job_families_deduped = ref_job_families.drop_duplicates()
print(ref_job_families.shape, ref_job_families_deduped.shape)

(91, 2) (89, 2)


In [74]:
# Step 1: Normalize casing and strip whitespace
ref_job_families["Job Title"] = ref_job_families["Job Title"].str.strip().str.lower()
ref_job_families["Category"] = ref_job_families["Category"].str.strip().str.lower()

# Step 2: Drop duplicates based only on those two columns
ref_job_families_deduped = ref_job_families.drop_duplicates(subset=["Category", "Job Title"])

print(f"Before: {ref_job_families.shape}, After: {ref_job_families_deduped.shape}")


Before: (91, 2), After: (89, 2)


In [75]:
row_count = ref_job_families_deduped.groupby(["Category", "Job Title"]).size().reset_index(name='count')
print(row_count[row_count['count'] > 1])  # Should be empty if deduped correctly

Empty DataFrame
Columns: [Category, Job Title, count]
Index: []
